Msc Data Science & AI  
IS71130A: Natural Language Processing (2023-24)- 
Assessment 1: Coursework Project (50%) 

**Asclepius-Synthetic-Clinical-Notes- Summarisation**

## **I. Introducion**

## **II. Ethics**

License: cc-by-nc-sa-4.0

Citation Information: 
@misc{kweon2023publicly,
    title={Publicly Shareable Clinical Large Language Model Built on Synthetic Clinical Notes},
    author={Sunjun Kweon and Junu Kim and Jiyoun Kim and Sujeong Im and Eunbyeol Cho and Seongsu Bae and Jungwoo Oh and Gyubok Lee and Jong Hak Moon and Seng Chan You and Seungjin Baek and Chang Hoon Han and Yoon Bin Jung and Yohan Jo and Edward Choi},
    year={2023},
    eprint={2309.00237},
    archivePrefix={arXiv},
    primaryClass={cs.CL}
}

## **III. Methods**

### **3.1 Data Acquisition**

In [1]:
%pip install importlib

Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install scispacy
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

  Using cached spacy-3.7.4-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.6 MB)
  Using cached thinc-8.2.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (937 kB)
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.12
    Uninstalling thinc-8.1.12:
      Successfully uninstalled thinc-8.1.12
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.4.1 requires spacy<3.5.0,>=3.4.0, but you have spacy 3.7.4 which is incompatible.
en-core-sci-sm 0.5.1 requires spacy<3.5.0,>=3.4.1, but you have spacy 3.7.4 which is incompatible.
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz (15.9 MB)
  Preparing metadata (

In [3]:
import importlib.util

# Check if NLTK is already installed
if importlib.util.find_spec("nltk") is None:
    # NLTK is not installed, so install it
    !pip install nltk
else:
    print("NLTK is already installed.")

# Check if textblob is already installed
if importlib.util.find_spec("textblob") is None:
    # textblob is not installed, so install it
    !pip install textblob
else:
    print("textblob is already installed.")

# Check if spacy is already installed
if importlib.util.find_spec("spacy") is None:
    # spacy is not installed, so install it
    !pip install spacy
else:
    print("spacy is already installed.")



NLTK is already installed.
textblob is already installed.
spacy is already installed.


In [4]:
!python -m spacy download en_core_web_sm

2024-03-18 23:35:45.182380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 23:35:48.314344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 23:35:48.314475: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 23:35:48.334344: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there m

In [5]:
# Dependicies and Requirements
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk import pos_tag
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
import networkx as nx
from tqdm import tqdm


# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') 

/mnt/data/skanda/mambaforge/envs/gpu/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-03-18 23:35:58.382255: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 23:35:59.574523: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-18 23:35:59.574682: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS

True

In [6]:
data = pd.read_csv('synthetic.csv')

### **3.2 EDA**

In [7]:
data.head()

,patient_id,note,question,answer,task
0,0,Discharge Summary:\n\nPatient: 60-year-old mal...,Can you provide a simplified paraphrase of the...,The healthcare team used a gradual approach to...,Paraphrasing
1,1,Discharge Summary:\n\nAdmission Date: [Insert ...,Which coreferences were resolved in the hospit...,The hospital course section resolved the coref...,Coreference Resolution
2,2,Hospital Course Summary:\n\nAdmission Date: [I...,What were the key improvements in the patient'...,"During the hospital course, the patient's medi...",Summarization
3,3,Discharge Summary:\n\nPatient: 69-year-old mal...,What roles did physical therapists have in the...,Physical therapists were responsible for ensur...,Relation Extraction
4,4,Discharge Summary:\n\nPatient Information:\n- ...,What manual airway clearance techniques were u...,The discharge summary stated that 1-2 physical...,Relation Extraction


In [8]:
filtered_data = data[data['task'] == 'Summarization']

In [9]:
filtered_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19756 entries, 2 to 158107
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patient_id  19756 non-null  int64 
 1   note        19756 non-null  object
 2   question    19756 non-null  object
 3   answer      19756 non-null  object
 4   task        19756 non-null  object
dtypes: int64(1), object(4)
memory usage: 926.1+ KB


In [10]:
filtered_data.isna().sum()

patient_id    0
note          0
question      0
answer        0
task          0
dtype: int64

In [11]:
filtered_data

,patient_id,note,question,answer,task
2,2,Hospital Course Summary:\n\nAdmission Date: [I...,What were the key improvements in the patient'...,"During the hospital course, the patient's medi...",Summarization
5,5,Discharge Summary:\n\nPatient: 52-year-old mal...,How did the patient's treatment for dysphagia ...,"During the patient's hospital stay, treatment ...",Summarization
10,11,Discharge Summary:\n\nPatient Name: [REDACTED]...,"Can you provide a summary of the treatment, ho...",The 45-year-old female patient with a history ...,Summarization
12,13,DISCHARGE SUMMARY:\n\nPatient Name: X\nMedical...,"Based on the given discharge summary, can you ...",The patient with a multifocal invasive mammary...,Summarization
18,20,Hospital Course:\n\nThe patient is a 78-year-o...,What are the key findings and diagnosis of the...,The key findings of the patient include abnorm...,Summarization
...,...,...,...,...,...
158074,166992,"Hospital Course:\n\nThe patient, a 90-year-old...",What was the diagnosis and treatment course fo...,The 90-year-old male patient was diagnosed wit...,Summarization
158077,166995,DISCHARGE SUMMARY\n\nPatient Information:\nNam...,Can you provide a concise summary of the disch...,The patient's chief complaint was extreme fati...,Summarization
158080,166999,Hospital Course: \n\nJK is a 35-year-old male ...,What was the response to clofarabine and the p...,The 35-year-old male patient with recurrent T-...,Summarization
158088,167008,DISCHARGE SUMMARY\n\nPatient Name: [redacted]\...,"What was the diagnosis, treatment, and recurre...","Based on the given discharge summary, the pati...",Summarization


In [12]:
text = filtered_data['note'].head(5)
print(text)

2     Hospital Course Summary:\n\nAdmission Date: [I...
5     Discharge Summary:\n\nPatient: 52-year-old mal...
10    Discharge Summary:\n\nPatient Name: [REDACTED]...
12    DISCHARGE SUMMARY:\n\nPatient Name: X\nMedical...
18    Hospital Course:\n\nThe patient is a 78-year-o...
Name: note, dtype: object


### **3.3 Text Cleaning**

### **3.4 Text Preprocessing**

In [13]:
class TextPreprocessor:
    def __init__(self):
        self.placeholder_pattern = r'\[Insert date\]|\[Insert Number\]|\[Patient\'s Name\]|Patient\'s Name:'
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()
        self.lemmatizer = WordNetLemmatizer()

    def clean_text(self, text_series):
        """
        Cleans the input text by performing several preprocessing steps: placeholder removal, case normalization,
        and whitespace normalization.

        :param text_series: A pandas Series containing the raw text.
        :return: A pandas Series containing the cleaned text.
        """
        cleaned_text = text_series.str.replace(self.placeholder_pattern, '', regex=True, case=False)
        cleaned_text = cleaned_text.str.lower()
        cleaned_text = cleaned_text.str.split().str.join(' ')
        return cleaned_text

    def preprocess_text(self, text):
        """
        Preprocesses the given text by performing several steps: sentence segmentation, word tokenization,
        stemming, lemmatization, stop word removal, and part-of-speech tagging.

        :param text: The input cleaned text to preprocess.
        :return: A dictionary containing various levels of processed text.
        """
        # Initial preprocessing steps
        sentences = sent_tokenize(text)
        words = [word_tokenize(sentence) for sentence in sentences]

        # Stemming and Lemmatization
        stemmed_words = [[self.stemmer.stem(word) for word in word_list] for word_list in words]
        lemmatized_words = [[self.lemmatizer.lemmatize(word) for word in word_list] for word_list in words]

        # Stop Word Removal
        filtered_sentences = [[word for word in word_list if word not in self.stop_words] for word_list in words]

        # Part-of-Speech Tagging (on filtered sentences to mimic real processing flow)
        flat_words = [word for sublist in filtered_sentences for word in sublist]
        pos_tags = nltk.pos_tag(flat_words)

        return {
            'sentences': sentences,
            'words': words,
            'stemmed_words': stemmed_words,
            'lemmatized_words': lemmatized_words,
            'filtered_sentences': filtered_sentences,
            'pos_tags': pos_tags
        }

# Create an instance of the TextPreprocessor class
text_preprocessor = TextPreprocessor()

# Clean the notes and add a new column 'cleaned_note' to the DataFrame
filtered_data['cleaned_note'] = text_preprocessor.clean_text(filtered_data['note'])

# Access the cleaned notes
print(filtered_data['cleaned_note'].iloc[0])

# Preprocess the cleaned text and add a new column 'preprocessed_text' to the DataFrame
filtered_data['preprocessed_text'] = filtered_data['cleaned_note'].apply(text_preprocessor.preprocess_text)

# Access and print the preprocessed text for the first row
print(filtered_data['preprocessed_text'].iloc[0])

/tmp/ipykernel_3935889/3656660417.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['cleaned_note'] = text_preprocessor.clean_text(filtered_data['note'])


hospital course summary: admission date: discharge date: patient: sex: male age: 57 years admission diagnosis: oxygen desaturation hospital course: the patient was admitted to the icu one week after a positive covid-19 result due to oxygen desaturation. physical therapy was initiated promptly after admission, which helped improve the patient's breathing frequency and oxygen saturation. the patient was guided to achieve a prone position resulting in a significant increase in oxygen saturation from 88% to 96%. the patient continued to receive intensive physical therapy, positioning, and oxygen therapy for the next few days. although there were challenges in achieving the prone position due to the patient's profoundly reduced respiratory capacity and high risk of symptom exacerbation, the medical team succeeded in implementing a safe and individualized approach. after three days with this regime, the patient was transferred to the normal ward, where physical therapists continued his rehab

{'sentences': ['hospital course summary: admission date: discharge date: patient: sex: male age: 57 years admission diagnosis: oxygen desaturation hospital course: the patient was admitted to the icu one week after a positive covid-19 result due to oxygen desaturation.', "physical therapy was initiated promptly after admission, which helped improve the patient's breathing frequency and oxygen saturation.", 'the patient was guided to achieve a prone position resulting in a significant increase in oxygen saturation from 88% to 96%.', 'the patient continued to receive intensive physical therapy, positioning, and oxygen therapy for the next few days.', "although there were challenges in achieving the prone position due to the patient's profoundly reduced respiratory capacity and high risk of symptom exacerbation, the medical team succeeded in implementing a safe and individualized approach.", 'after three days with this regime, the patient was transferred to the normal ward, where physical

/tmp/ipykernel_3935889/3656660417.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['preprocessed_text'] = filtered_data['cleaned_note'].apply(text_preprocessor.preprocess_text)


### **3.5 Feature Extraction**

In [14]:
# Load the en_core_sci_sm model
nlp = spacy.load("en_core_sci_sm")

# Add the "ner" component to the pipeline
ner = nlp.get_pipe("ner")

def extract_features(text):
    """
    Extracts medical entities and TF-IDF features from the input text.

    :param text: The input text to extract features from.
    :return: A tuple containing the list of medical entities and the TF-IDF vector.
    """
    doc = nlp(text)

    # Extract medical entities
    entities = []
    for ent in doc.ents:
        entities.append((ent.text, ent.label_))

    # Compute TF-IDF vector
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_vector = tfidf_vectorizer.fit_transform([text])

    return entities, tfidf_vector

# Process a sample clinical note
# sample_note = filtered_data['cleaned_note'].iloc[0]
# entities, tfidf_vector = extract_features(sample_note)

# print("Medical Entities:")
# print(entities)

# print("\nTF-IDF Vector:")
# print(tfidf_vector)

In [15]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

def combine_features(text, entities, tfidf_vector):
    """
    Combines the medical entities and TF-IDF features into a single feature vector.

    :param text: The input text.
    :param entities: The list of medical entities extracted from the text.
    :param tfidf_vector: The TF-IDF vector for the text.
    :return: A combined feature vector representing both medical entities and TF-IDF features.
    """
    # Extract unique entity labels
    unique_labels = set([label for _, label in entities])

    # One-hot encode the entity labels
    entity_labels = [label for _, label in entities]
    one_hot_encoder = OneHotEncoder(sparse=False)
    entity_features = one_hot_encoder.fit_transform([[label] for label in entity_labels])

    # Sum the entity features along the rows
    combined_entity_features = np.sum(entity_features, axis=0, keepdims=True)

    # Combine the entity features and TF-IDF vector
    combined_features = np.concatenate((combined_entity_features, tfidf_vector.toarray()), axis=1)

    return combined_features

### **3.5 Model Building**

In [16]:
# def textrank_summarize(text, num_sentences):
#     # Load the spaCy English model
#     nlp = spacy.load("en_core_sci_sm")

#     # Tokenize the text into sentences
#     doc = nlp(text)
#     sentences = [sent.text.capitalize() for sent in doc.sents]

#     # Create a similarity matrix between sentences
#     similarity_matrix = nx.Graph()
#     for i in range(len(sentences)):
#         for j in range(i+1, len(sentences)):
#             similarity = nlp(sentences[i]).similarity(nlp(sentences[j]))
#             similarity_matrix.add_edge(i, j, weight=similarity)

#     # Apply PageRank to compute sentence importance scores
#     scores = nx.pagerank(similarity_matrix)

#     # Check if there are any scores assigned to the sentences
#     if not scores:
#         return ""  # Return an empty string if no scores are assigned

#     # Sort sentences by their importance scores
#     ranked_sentences = sorted([(scores[i], sent) for i, sent in enumerate(sentences)], reverse=True)

#     # Extract the top-ranked sentences as the summary
#     summary = " ".join([sent for _, sent in ranked_sentences[:num_sentences]])

#     return summary

# def process_and_summarize(text):
#     # Clean the text
#     cleaned_text = text_preprocessor.clean_text(pd.Series(text))[0]
    
#     # Preprocess the text
#     preprocessed_text = text_preprocessor.preprocess_text(cleaned_text)
    
#     # Extract features (if required)
#     # entities = extract_named_entities(cleaned_text)
    
#     # Apply the TextRank summarization model
#     summary = textrank_summarize(cleaned_text, num_sentences=3)
    
#     return summary

# # Apply the process_and_summarize function to the 'note' column with a progress bar
# tqdm.pandas(desc="Summarizing notes")
# filtered_data['summary'] = filtered_data['note'].progress_apply(process_and_summarize)

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer

def textrank_summarize(text, num_sentences):
    # Load the spaCy English model
    nlp = spacy.load("en_core_web_sm")

    # Tokenize the text into sentences
    doc = nlp(text)
    sentences = [sent.text.capitalize() for sent in doc.sents]

    # Precompute sentence embeddings using TF-IDF
    vectorizer = TfidfVectorizer()
    sentence_embeddings = vectorizer.fit_transform(sentences)

    # Create a similarity matrix between sentences
    similarity_matrix = nx.Graph()
    for i in range(len(sentences)):
        for j in range(i+1, len(sentences)):
            similarity = sentence_embeddings[i].dot(sentence_embeddings[j].T).toarray()[0][0]
            similarity_matrix.add_edge(i, j, weight=similarity)

    # Apply PageRank to compute sentence importance scores
    scores = nx.pagerank(similarity_matrix)

    # Check if there are any scores assigned to the sentences
    if not scores:
        return ""  # Return an empty string if no scores are assigned

    # Sort sentences by their importance scores
    ranked_sentences = sorted([(scores[i], sent) for i, sent in enumerate(sentences)], reverse=True)

    # Extract the top-ranked sentences as the summary
    summary = " ".join([sent for _, sent in ranked_sentences[:num_sentences]])

    return summary

def process_and_summarize(text):
    # Clean the text
    cleaned_text = text_preprocessor.clean_text(pd.Series(text))[0]
    
    # Preprocess the text
    preprocessed_text = text_preprocessor.preprocess_text(cleaned_text)
    
    # Apply the TextRank summarization model
    summary = textrank_summarize(cleaned_text, num_sentences=3)
    
    return summary

tqdm.pandas(desc="Summarizing notes")
filtered_data['summary'] = filtered_data['note'].progress_apply(process_and_summarize)

Summarizing notes:  13%|█▎        | 2530/19756 [20:57<2:13:21,  2.15it/s]

In [ ]:
# # fuzzy clustering
# import numpy as np
# from skfuzzy import cmeans, membership

# def apply_fuzzy_clustering(combined_features, n_clusters=3, m=2, error=0.005, maxiter=1000):
#     """
#     Applies fuzzy c-means clustering on the combined features.

#     :param combined_features: The combined feature vector.
#     :param n_clusters: The number of clusters to create (default: 3).
#     :param m: The fuzziness parameter (default: 2).
#     :param error: The stopping criterion for the fuzzy c-means algorithm (default: 0.005).
#     :param maxiter: The maximum number of iterations for the fuzzy c-means algorithm (default: 1000).
#     :return: A tuple containing the cluster centers, cluster membership values, and the number of iterations.
#     """
#     # Apply fuzzy c-means clustering
#     cntr, u, _, _, _, _, _, _= cmeans(
#         combined_features.T, n_clusters, m, error=error, maxiter=maxiter, init=None
#     )

#     # Calculate the membership values for each sample
#     membership_values = np.argmax(u, axis=0)

#     return cntr, u, membership_values

# # Process a sample clinical note
# sample_note = filtered_data['cleaned_note'].iloc[0]
# entities, tfidf_vector = extract_features(sample_note)
# combined_features = combine_features(sample_note, entities, tfidf_vector)

# # Apply fuzzy clustering
# n_clusters = 3
# cluster_centers, membership_values, membership_labels = apply_fuzzy_clustering(combined_features, n_clusters=n_clusters)

# print(f"Cluster Centers:\n{cluster_centers}")
# print(f"\nMembership Values:\n{membership_values}")
# print(f"\nMembership Labels:\n{membership_labels}")

#### **BLEU and ROUGE score for NLP Evaluation**

In [ ]:
!pip install rouge

In [ ]:
from rouge import Rouge

rouge = Rouge()
scores = rouge.get_scores(filtered_data['summary'], filtered_data['cleaned_note'], avg=True)
print("ROUGE scores:", scores)

KeyError: 'summary'

### **3.6 Model Evaluation**

## **IV. Conclusion**

### References